# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
#Ignore copy warning
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = os.path.join('..', 'WeatherPy', 'Resources', 'Cities.csv')

weather_df = pd.read_csv(weather_file)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Aksu,41.1231,80.2644,85.64,20,0,9.04,CN,1658293787
1,Phan Thiet,10.9333,108.1000,83.66,73,100,9.35,VN,1658293787
2,Alberton,46.8168,-64.0654,65.05,87,36,20.58,CA,1658293787
3,Vaini,-21.2000,-175.2000,73.56,100,20,11.50,TO,1658293787
4,Nome,64.5011,-165.4064,46.47,71,100,5.75,US,1658293536
...,...,...,...,...,...,...,...,...,...
548,Pirganj,25.8523,88.3618,88.48,69,99,8.34,BD,1658293960
549,Kavieng,-2.5744,150.7967,83.34,71,100,5.23,PG,1658293960
550,Tambopata,-12.7333,-69.1833,75.33,77,100,2.01,PE,1658293960
551,Fenelon Falls,44.5334,-78.7496,69.57,80,90,5.19,CA,1658293961


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configuring gmaps
gmaps.configure(api_key=g_key)

#Make locations and weights to configure
locations = weather_df[['Lat', 'Lng']]

#Change humidity from string to float to avoid ValueError
weights = weather_df['Humidity'].astype(float)
max_humidity = max(weather_df["Humidity"])

max_humidity

100

In [4]:
#adding heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity = max_humidity, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Making new weather df with weather criteria, using copy to avoid altering original df
new_weather_df = weather_df.copy()

new_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Aksu,41.1231,80.2644,85.64,20,0,9.04,CN,1658293787
1,Phan Thiet,10.9333,108.1000,83.66,73,100,9.35,VN,1658293787
2,Alberton,46.8168,-64.0654,65.05,87,36,20.58,CA,1658293787
3,Vaini,-21.2000,-175.2000,73.56,100,20,11.50,TO,1658293787
4,Nome,64.5011,-165.4064,46.47,71,100,5.75,US,1658293536
...,...,...,...,...,...,...,...,...,...
548,Pirganj,25.8523,88.3618,88.48,69,99,8.34,BD,1658293960
549,Kavieng,-2.5744,150.7967,83.34,71,100,5.23,PG,1658293960
550,Tambopata,-12.7333,-69.1833,75.33,77,100,2.01,PE,1658293960
551,Fenelon Falls,44.5334,-78.7496,69.57,80,90,5.19,CA,1658293961


In [6]:
#Making new df with desired weather criteria: Max Temp between 70 and 80, Cloudiness at 0, and Wind Speed less than 10
criteria_weather_df = new_weather_df.loc[(new_weather_df["Max Temp"] < 80) & (new_weather_df["Max Temp"] > 70) & 
                                         (new_weather_df["Cloudiness"] == 0) & (new_weather_df["Wind Speed"] < 10),:] 

#Using inplace to update df
criteria_weather_df.dropna(inplace=True)
criteria_weather_df.reset_index(drop=True, inplace=True)

In [7]:
criteria_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Martinez,38.0194,-122.1341,73.26,67,0,5.75,US,1658293530
1,Hamilton,39.1834,-84.5333,77.81,85,0,6.91,US,1658293689
2,Jamestown,42.0970,-79.2353,74.79,86,0,0.00,US,1658293508
3,At-Bashi,41.1702,75.8106,70.45,28,0,1.72,KG,1658293799
4,Tautira,-17.7333,-149.1500,76.64,67,0,3.87,PF,1658293806
5,Hilo,19.7297,-155.0900,76.73,81,0,3.44,US,1658293685
6,Linxia Chengguanzhen,35.6003,103.2064,78.12,36,0,3.49,CN,1658293815
7,Aleksandrov Gay,50.1470,48.5704,73.85,36,0,8.21,RU,1658293816
8,Lapeer,43.0514,-83.3188,77.49,60,0,6.91,US,1658293617
9,Bainbridge Island,47.6262,-122.5212,71.46,73,0,4.61,US,1658293662


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = criteria_weather_df

#Making new empty column to insert values into
hotel_df["Hotel Name"] = " "


#Importing/Setting parameters dictionary with 5000 meters
params = {"radius": 5000,
          "type": "hotels",
          "keyword": "hotel",
          "key": g_key}

#Setting base_url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [9]:
#Initialize loop to make markers for hotels in all cities by using iterrows
for index, row in hotel_df.iterrows():
    #Get all latitude, longitude, and name info for cities (*class 6.3)
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    #Add Lat/Lng back to parameters dictionary
    params["location"] = f"{lat},{lng}"
    
    #Making request for base url and converting to json
    hotel_locations = requests.get(base_url, params=params).json()
    
    #Adding data from json response to new column with Hotel Names
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_locations['results'][0]['name']
        
    except: 
        print('Unable to locate hotel at', row['City'], '... skipping')

Unable to locate hotel at Tautira ... skipping
Unable to locate hotel at Aleksandrov Gay ... skipping
Unable to locate hotel at Aripuanã ... skipping


In [11]:
#Using inplace to update df
hotel_df.dropna(inplace=True)

#Deleting empty rows without hotels 
hotel_df = hotel_df[hotel_df["Hotel Name"] != " "]
hotel_df.reset_index(drop=True, inplace=True)

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Martinez,38.0194,-122.1341,73.26,67,0,5.75,US,1658293530,"Holiday Inn Express Benicia, an IHG Hotel"
1,Hamilton,39.1834,-84.5333,77.81,85,0,6.91,US,1658293689,Graduate Cincinnati
2,Jamestown,42.0970,-79.2353,74.79,86,0,0.00,US,1658293508,Chautauqua Harbor Hotel
3,At-Bashi,41.1702,75.8106,70.45,28,0,1.72,KG,1658293799,Hostel At-Bashy
4,Hilo,19.7297,-155.0900,76.73,81,0,3.44,US,1658293685,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
5,Linxia Chengguanzhen,35.6003,103.2064,78.12,36,0,3.49,CN,1658293815,Baohan Hotel
6,Lapeer,43.0514,-83.3188,77.49,60,0,6.91,US,1658293617,"Holiday Inn Express Lapeer, an IHG Hotel"
7,Bainbridge Island,47.6262,-122.5212,71.46,73,0,4.61,US,1658293662,The Marshall Suites
8,Cayenne,4.9333,-52.3333,75.24,1,0,0.00,GF,1658293855,Grand Hotel Montabo
9,Saint-Pierre,-21.3393,55.4781,73.29,54,0,9.22,RE,1658293861,"Villa Delisle, Hotel & Spa"


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
figure_layout = {'width': "400px", 'height': "300px", 'border': "1px solid black", 'padding':"1px", 'margin': "0 auto 0 auto"}

fig = gmaps.figure(layout=figure_layout)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

In [14]:
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [14]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


Note: you may need to restart the kernel to use updated packages.
